### Import Libraries

In [1]:
# Tensorflow
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers, models
import tensorflow_hub as hub

import os
import warnings
warnings.filterwarnings("ignore")

### Rename Dataset

In [3]:
# def rename_images_in_directory(directory):
#     for subdir, _, files in os.walk(directory):
#         for i, file in enumerate(files):
#             file_ext = os.path.splitext(file)[1]
#             class_name = os.path.basename(subdir)
#             new_name = f"{class_name}_{i+1}{file_ext}"
#             old_file = os.path.join(subdir, file)
#             new_file = os.path.join(subdir, new_name)
#             os.rename(old_file, new_file)

# # Rename images in train and test directories
# train_dir = 'dataset/train'
# test_dir = 'dataset/test'

# rename_images_in_directory(train_dir)
# rename_images_in_directory(test_dir)

### Load and Preprocess Data

In [15]:
# Define paths
train_dir = 'dataset/train'
test_dir = 'dataset/test'

# Load datasets
batch_size = 32
img_height = 224
img_width = 224

train_dataset = image_dataset_from_directory(train_dir,
                                             shuffle=True,
                                             batch_size=batch_size,
                                             image_size=(img_height, img_width))

test_dataset = image_dataset_from_directory(test_dir,
                                            shuffle=True,
                                            batch_size=batch_size,
                                            image_size=(img_height, img_width))

Found 673 files belonging to 17 classes.
Found 169 files belonging to 17 classes.


### Build, Train, and Evaluate the Model

In [16]:
# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=(img_height, img_width, 3),
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False

# Add classification head
num_classes = len(train_dataset.class_names)

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(17)
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d_1  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_1 (Dense)             (None, 17)                21777     
                                                                 
Total params: 2279761 (8.70 MB)
Trainable params: 21777 (85.07 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [17]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

epochs = 10
history = model.fit(train_dataset, validation_data=test_dataset, epochs=epochs)

Epoch 1/10
Cause: Unable to locate the source code of <function Model.make_train_function.<locals>.train_function at 0x0000021B2587D760>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: Unable to locate the source code of <function Model.make_train_function.<locals>.train_function at 0x0000021B2587D760>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. O

In [18]:
loss, accuracy = model.evaluate(test_dataset)
print(f'Test accuracy: {accuracy}')

6/6 [==============================] - 1s 223ms/step - loss: 1.3179 - accuracy: 0.5858
Test accuracy: 0.5857987999916077


### Convert the Model to TFLite:



In [19]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model converted to TFLite and saved as 'model.tflite'")

Cause: Unable to locate the source code of <function trace_model_call.<locals>._wrapped_model at 0x0000021B283EC540>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: Unable to locate the source code of <function trace_model_call.<locals>._wrapped_model at 0x0000021B283EC540>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get 

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('signature_function', 'signature_key'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('signature_function', 'signature_key'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('signature_function', 'signature_key'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


INFO:tensorflow:Assets written to: C:\Users\david\AppData\Local\Temp\tmpc_04hu9b\assets


INFO:tensorflow:Assets written to: C:\Users\david\AppData\Local\Temp\tmpc_04hu9b\assets


Model converted to TFLite and saved as 'model.tflite'
